In [1]:
import math
import torch

from matplotlib import pyplot as plt
%matplotlib inline

import os
import argparse
import pandas as pd
from pandas import factorize
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
#from sklearn.model_selection import KFold

import gpytorch
import itertools

import warnings
warnings.filterwarnings('ignore')


In [2]:
from m3gpr.config import get_cfg_defaults_all
from m3gpr.load_data import Data_set
from m3gpr.models import singleGP_gpytorch,rf_sklearn,singleGP_gpytorch_train,singleGP_gpytorch_reference,MultitaskGPModel,MTMOGP
from m3gpr.train import CV_Trainer

In [3]:
cfg = get_cfg_defaults_all()
os.chdir('/Users/chenya68/Documents/GitHub/BFO')
parser = argparse.ArgumentParser(description="Configure files")
parser.add_argument("--cfg", required=True, help="path to config file", type=str)
#args = parser.parse_args(['--cfg', 'data/harpoon/harpoon_dataset.yaml'])
args = parser.parse_args(['--cfg', 'bfo-data/fdt_data/fdt_dataset.yaml'])
cfg.merge_from_file(args.cfg)

cfg.freeze()
setup_data = Data_set(cfg, 1)
print(cfg.MODEL.MODEL_NAME)
print(cfg.MODEL.Y_SCALE)

#df_x, df_y_wide, df_y_long = setup_data.Load_data_set()
df_x, df_y,ls_X_train,ls_X_test,ls_y_train,ls_y_test, obj_y_scaler= setup_data.Load_data_set()
print(df_x.shape)
print(df_y.shape)
#print(df_y_wide.shape)
#print(df_y_long.shape)

singleGP
y-stand
(454, 55)
(454, 5)


In [4]:
df_y_corr = df_y.corr()
df_y_corr.to_csv(cfg.PATH.DATA_PATH + 'df_y_corr.csv')
print(df_y_corr)

                        HARDNESS  FRIABILITY  Drug_content  \
HARDNESS                1.000000    0.091377      0.048112   
FRIABILITY              0.091377    1.000000      0.053990   
Drug_content            0.048112    0.053990      1.000000   
Water_absorption_ratio  0.003404    0.041980     -0.005044   
DISINTEGRATION_TIME    -0.095159   -0.108785     -0.065873   

                        Water_absorption_ratio  DISINTEGRATION_TIME  
HARDNESS                              0.003404            -0.095159  
FRIABILITY                            0.041980            -0.108785  
Drug_content                         -0.005044            -0.065873  
Water_absorption_ratio                1.000000            -0.183772  
DISINTEGRATION_TIME                  -0.183772             1.000000  


In [5]:
import numpy as np

def correlation_to_covariance(corr_matrix, std_devs):
    """
    Convert a correlation matrix to a covariance matrix.

    Parameters:
        corr_matrix (np.ndarray): Square correlation matrix (n x n).
        std_devs (np.ndarray): 1D array of standard deviations (length n).

    Returns:
        np.ndarray: Covariance matrix (n x n).
    """
    # Validate inputs
    corr_matrix = np.array(corr_matrix, dtype=float)
    std_devs = np.array(std_devs, dtype=float)

    if corr_matrix.shape[0] != corr_matrix.shape[1]:
        raise ValueError("Correlation matrix must be square.")
    if corr_matrix.shape[0] != std_devs.shape[0]:
        raise ValueError("Length of std_devs must match matrix size.")
    if not np.allclose(corr_matrix, corr_matrix.T, atol=1e-8):
        raise ValueError("Correlation matrix must be symmetric.")
    if np.any(np.diag(corr_matrix) != 1):
        raise ValueError("Diagonal elements of correlation matrix must be 1.")

    # Create diagonal matrix of standard deviations
    D = np.diag(std_devs)

    # Covariance = D * Corr * D
    cov_matrix = D @ corr_matrix @ D
    return cov_matrix


# Example usage
corr = df_y_corr.values
std_devs = df_y.std().to_list()

cov = correlation_to_covariance(corr, std_devs)
cov = torch.nn.Parameter(torch.Tensor(cov))
print("Covariance Matrix:\n", cov)


Covariance Matrix:
 Parameter containing:
tensor([[ 9.9152e-01,  2.6691e-02,  2.3423e+00,  1.4601e-01, -3.4544e+00],
        [ 2.6691e-02,  8.6049e-02,  7.7433e-01,  5.3046e-01, -1.1634e+00],
        [ 2.3423e+00,  7.7433e-01,  2.3904e+03, -1.0624e+01, -1.1741e+02],
        [ 1.4601e-01,  5.3046e-01, -1.0624e+01,  1.8556e+03, -2.8860e+02],
        [-3.4544e+00, -1.1634e+00, -1.1741e+02, -2.8860e+02,  1.3290e+03]],
       requires_grad=True)


## Single GP Cross-Validation

In [ ]:
if True:

        model_name = cfg.MODEL.MODEL_NAME
        n_cv = cfg.MODEL.N_CV#cfg.MODEL.N_CV is number of cv
        num_outputs = cfg.DATA.NUM_OUTPUTS
        if cfg.MODEL.SPLIT == 'by-task':
            num_tasks = cfg.DATA.NUM_TASKS
        else:
            num_tasks = 1
        num_total_output = num_outputs*num_tasks
        num_cols = num_total_output*n_cv
        y_scale=cfg.MODEL.Y_SCALE
        metric_row = 3

        
        ls_model_from_combs = [[] for _ in range(num_total_output)]
        ls_init_len = [2.0,3.0,4.0]
        ls_lr = [0.1,0.15,0.2,0.25]
        ls_n_iter = [200,300]
        combs = list(itertools.product(ls_init_len,ls_lr,ls_n_iter))

        mean_arr_cv_mae = np.zeros((2,len(combs))) #train,test
        mean_arr_cv_r2 = np.zeros((2,len(combs))) #train,test
        mean_arr_cv_err = np.zeros((2,len(combs))) #train,test

        ls_arr_cv_mae = [np.zeros((2,len(combs))) for _ in range(num_total_output)]
        ls_arr_cv_r2 = [np.zeros((2,len(combs))) for _ in range(num_total_output)]
        ls_arr_cv_err = [np.zeros((2,len(combs))) for _ in range(num_total_output)]

        
        for i in range(0,len(combs)):
            init_len,lr,n_iter = combs[i]
            arr_train_metrics = np.zeros((11,num_cols))
            arr_test_metrics = np.zeros((11,num_cols))
            
            for batch_ind in range(n_cv):
                t_train_x = ls_X_train[batch_ind]
                t_test_x = ls_X_test[batch_ind]
                
                #for j in range(self.num_outputs):
                for j in range(num_total_output):
                    a = batch_ind + n_cv*j
                    t_train_y = ls_y_train[batch_ind][:,j].flatten()
                    t_test_y = ls_y_test[batch_ind][:,j].flatten()

                    train_mean,train_lower,train_upper,model = singleGP_gpytorch_train(t_train_x,t_train_y,training_iter = n_iter,init_len_scale = init_len,my_lr = lr)
                    #print(train_mean)
                    ls_model_from_combs[j].append(model)
                    
                    # Set into eval mode
                    model.eval()
                    model.likelihood.eval()

                    with torch.no_grad(), gpytorch.settings.fast_pred_var():
                        test_pred = model.likelihood(model(t_test_x))   
                        test_mean = test_pred.mean
                        test_lower, test_upper = test_pred.confidence_region()

                    if y_scale=='no-y-scale':
                        arr_train_y = t_train_y.detach().numpy()
                        arr_test_y = t_test_y.detach().numpy()
                        arr_test_mean = test_mean.detach().numpy()
                        arr_test_lower = test_lower.detach().numpy()
                        arr_test_upper = test_upper.detach().numpy()

                        arr_train_mean = train_mean.detach().numpy()
                        arr_train_lower = train_lower.detach().numpy()
                        arr_train_upper = train_upper.detach().numpy()
                    else:
                        y_scaler = obj_y_scaler[batch_ind]
                        tmp_mean = y_scaler.mean_
                        tmp_scale = y_scaler.scale_

                        arr_train_y = t_train_y.detach().numpy()
                        arr_test_y = t_test_y.detach().numpy()
                        arr_test_mean = test_mean.detach().numpy()
                        arr_test_lower = test_lower.detach().numpy()
                        arr_train_mean = train_mean.detach().numpy()
                        arr_train_lower = train_lower.detach().numpy()

                        arr_train_y = arr_train_y*tmp_scale[j] + tmp_mean[j]
                        arr_test_mean = arr_test_mean*tmp_scale[j] + tmp_mean[j]
                        arr_test_lower = arr_test_lower*tmp_scale[j] + tmp_mean[j]
                        arr_train_mean = arr_train_mean*tmp_scale[j] + tmp_mean[j]
                        arr_train_lower = arr_train_lower*tmp_scale[j] + tmp_mean[j]

                    y_true = arr_test_y
                    y_pred = arr_test_mean
                    y_train = arr_train_y
                    mean_train = arr_train_mean
                    std_train = arr_train_mean - arr_train_lower
                    std_test = arr_test_mean - arr_test_lower

                    train_comp = np.concatenate((y_train.reshape(-1,1),mean_train.reshape(-1,1)),axis = 1)
                    df_train_comp = pd.DataFrame(train_comp,columns = ['true','pred'])
                    df_train_comp['upper'] = df_train_comp['pred'] + std_train.squeeze()
                    df_train_comp['lower'] = df_train_comp['pred'] - std_train.squeeze()
                    df_train_comp['mode'] = 'train'
    
                    test_comp = np.concatenate((y_true.reshape(-1,1),y_pred.reshape(-1,1)),axis = 1)
                    df_test_comp = pd.DataFrame(test_comp,columns = ['true','pred'])
                    df_test_comp['upper'] = df_test_comp['pred'] + std_test.squeeze()
                    df_test_comp['lower'] = df_test_comp['pred'] - std_test.squeeze()
                    df_test_comp['mode'] = 'test'
    

                    arr_test_metrics[0,a] = metrics.mean_absolute_error(y_true, y_pred)
                    arr_test_metrics[1,a] = metrics.median_absolute_error(y_true, y_pred)
                    arr_test_metrics[2,a] = metrics.mean_squared_error(y_true, y_pred)
                    arr_test_metrics[3,a] = metrics.root_mean_squared_error(y_true, y_pred)
                    arr_test_metrics[4,a] = metrics.mean_absolute_percentage_error(y_true, y_pred)
                    arr_test_metrics[5,a] = metrics.max_error(y_true, y_pred)
                    arr_test_metrics[6,a] = metrics.explained_variance_score(y_true, y_pred)
                    arr_test_metrics[7,a] = metrics.r2_score(y_true, y_pred)
                    arr_test_metrics[8,a] = np.mean(std_test)
                    arr_test_metrics[9,a] = np.min(std_test)
                    arr_test_metrics[10,a] = np.max(std_test)

                    arr_train_metrics[0,a] = metrics.mean_absolute_error(y_train, mean_train)
                    arr_train_metrics[1,a] = metrics.median_absolute_error(y_train, mean_train)
                    arr_train_metrics[2,a] = metrics.mean_squared_error(y_train, mean_train)
                    arr_train_metrics[3,a] = metrics.root_mean_squared_error(y_train, mean_train)
                    arr_train_metrics[4,a] = metrics.mean_absolute_percentage_error(y_train, mean_train)
                    arr_train_metrics[5,a] = metrics.max_error(y_train, mean_train)
                    arr_train_metrics[6,a] = metrics.explained_variance_score(y_train, mean_train)
                    arr_train_metrics[7,a] = metrics.r2_score(y_train, mean_train)
                    arr_train_metrics[8,a] = np.mean(std_train)
                    arr_train_metrics[9,a] = np.min(std_train)
                    arr_train_metrics[10,a] = np.max(std_train)
                    arr_train_metrics[8,a] = np.mean(std_train)
                    arr_train_metrics[9,a] = np.min(std_train)
                    arr_train_metrics[10,a] = np.max(std_train)
      
                    ls_arr_cv_mae[j][0,i] = np.mean(arr_train_metrics[metric_row,n_cv*j:n_cv*(j+1)])
                    ls_arr_cv_mae[j][1,i] = np.mean(arr_test_metrics[metric_row,n_cv*j:n_cv*(j+1)])

                    ls_arr_cv_r2[j][0,i] = np.mean(arr_train_metrics[6,n_cv*j:n_cv*(j+1)])
                    ls_arr_cv_r2[j][1,i] = np.mean(arr_test_metrics[6,n_cv*j:n_cv*(j+1)])

                    ls_arr_cv_err[j][0,i] = np.mean(arr_train_metrics[8,n_cv*j:n_cv*(j+1)])
                    ls_arr_cv_err[j][1,i] = np.mean(arr_test_metrics[8,n_cv*j:n_cv*(j+1)])

            mean_arr_cv_mae[0,i] = np.mean(arr_train_metrics[metric_row,:])
            mean_arr_cv_mae[1,i] = np.mean(arr_test_metrics[metric_row,:])

            mean_arr_cv_r2[0,i] = np.mean(arr_train_metrics[6,:])
            mean_arr_cv_r2[1,i] = np.mean(arr_test_metrics[6,:])

            mean_arr_cv_err[0,i] = np.mean(arr_train_metrics[8,:])
            mean_arr_cv_err[1,i] = np.mean(arr_test_metrics[8,:])

In [ ]:
if True:
    sgp_cv_trainer = CV_Trainer(cfg,ls_X_train,ls_X_test,ls_y_train,ls_y_test,obj_y_scaler,cov)
    combs,ls_model_from_combs,ls_arr_cv_mae,ls_arr_cv_r2,ls_arr_cv_err,mean_arr_cv_mae,mean_arr_cv_r2,mean_arr_cv_err = sgp_cv_trainer.set_up_sgp_cv()
    
    if cfg.MODEL.SPLIT == 'by-task':
        num_tasks = cfg.DATA.NUM_TASKS
    else:
        num_tasks = 1
    
    #num_tasks = cfg.DATA.NUM_TASKS
    num_outputs = cfg.DATA.NUM_OUTPUTS
    num_total_outputs = num_outputs*num_tasks
    arr_metrics = np.zeros((3,1+num_total_outputs)) #row:mae, r2, avg_std; col: total mean, task1(output1), task2(output2),...

    print('number of tasks that are being compared: ',num_total_outputs)

    best_idx = np.argmin(mean_arr_cv_mae[1,:])
    #best_idx = np.argmax(mean_arr_cv_r2[1,:])

    print('best_idx',best_idx)
    print('best combinations: ',combs[best_idx])

    arr_metrics[0,0] = np.min(mean_arr_cv_mae[1,:])
    arr_metrics[1,0] = np.max(mean_arr_cv_r2[1,:])
    arr_metrics[2,0] = mean_arr_cv_err[1,best_idx]

    print('best mean mae %.3f' % arr_metrics[0,0])
    print('best mean r2 %.3f' % arr_metrics[1,0])
    print('best mean err %.3f' % arr_metrics[2,0])
    
    cols_target_new = ['total']
    for task_ind in range(num_tasks):
        for out_ind in range(num_outputs):
            j = out_ind + num_outputs*task_ind
    #for j in range(num_total_outputs):
            j_name = 'Task_'+str(task_ind+1) + '_Output_'+str(out_ind+1)
            cols_target_new.append(j_name)
            arr_metrics[0,j+1] = ls_arr_cv_mae[j][1,best_idx]
            arr_metrics[1,j+1] = ls_arr_cv_r2[j][1,best_idx]
            arr_metrics[2,j+1] = ls_arr_cv_err[j][1,best_idx]
            print('best mae of target %d is %.3f' %(j,ls_arr_cv_mae[j][1,best_idx]))
            print('best r2 of target %d is %.3f' %(j,ls_arr_cv_r2[j][1,best_idx]))
            print('best err of target %d is %.3f' % (j,ls_arr_cv_err[j][1,best_idx]))
    
    df_test_metrics = pd.DataFrame(np.round(arr_metrics,3),columns = cols_target_new, 
                               index = [cfg.MODEL.METRIC,'R2','AVG_STD'])
    print(df_test_metrics)
    
    #output_path = '/'.join([cfg.PATH.RESULT,cfg.MISC.DATA_NAME,cfg.MODEL.MODEL_NAME,cfg.MODEL.SPLIT,cfg.MISC.DATE])
    df_test_metrics.to_csv(setup_data.output_path+'/df_test_metrics.csv')

## MO Cross-Validation

In [7]:
num_outputs = 5
mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=num_outputs
        )
x = ls_X_train[0]
mean_x = mean_module(x)
mean_x.shape

torch.Size([62, 5])

In [13]:
output_kernel = gpytorch.kernels.IndexKernel(num_tasks=num_outputs, rank = 1)
output_x = output_kernel.covar_matrix
data_kernel = gpytorch.kernels.MaternKernel()
data_x = data_kernel(x)
print(data_x.shape)
covar_x = gpytorch.lazy.KroneckerProductLazyTensor(data_x, output_x)
print(covar_x.shape)
result = gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)
result


torch.Size([62, 62])
torch.Size([310, 310])


MultitaskMultivariateNormal(mean shape: torch.Size([62, 5]))

In [ ]:
mo_cv_trainer = CV_Trainer(cfg,ls_X_train,ls_X_test,ls_y_train,ls_y_test,obj_y_scaler,cov)
combs,ls_model_from_combs,ls_arr_cv_mae,ls_arr_cv_r2,ls_arr_cv_err,mean_arr_cv_mae,mean_arr_cv_r2,mean_arr_cv_err = mo_cv_trainer.set_up_cv_model()

In [8]:
model = ls_model_from_combs[0]
output_kern = model.covar_module
covar_factor = output_kern.task_covar_module.covar_factor
print('W W^T + diag(v) is the output kernel covariance matrix: ')
B = output_kern.task_covar_module._eval_covar_matrix().detach().numpy()
print(B)
"""
print('---Manually calculating covariance matrix---')
W = covar_factor.detach().numpy()
B2 = np.dot(W,W.T)+np.diag(output_kern.task_covar_module.raw_var.detach().numpy())
print('W W^T + diag(v) is the output kernel covariance matrix: ')
print(B2)
"""

print('Correlation matrix between outputs')
C = (B/np.sqrt(np.diag(B))).T/np.sqrt(np.diag(B))
print(C)


W W^T + diag(v) is the output kernel covariance matrix: 
[[ 5.4112635e+00  2.4229001e-02  9.3053007e+00 -3.8525000e+00
  -1.6408001e-01]
 [ 2.4229001e-02  1.4141234e+00 -1.7058998e+00 -2.9240999e-01
  -3.2570000e+00]
 [ 9.3053007e+00 -1.7058998e+00  2.4771016e+03 -9.1609998e+02
   1.3841000e+02]
 [-3.8525000e+00 -2.9240999e-01 -9.1609998e+02  1.3570074e+03
  -8.3879986e+00]
 [-1.6408001e-01 -3.2570000e+00  1.3841000e+02 -8.3879986e+00
   1.6869198e+03]]
Correlation matrix between outputs
[[ 1.0000001   0.00875876  0.08037282 -0.04495751 -0.00171735]
 [ 0.00875876  1.0000001  -0.02882292 -0.00667509 -0.06668481]
 [ 0.08037282 -0.02882292  1.         -0.4996661   0.0677093 ]
 [-0.04495751 -0.00667509 -0.49966606  1.         -0.00554396]
 [-0.00171735 -0.0666848   0.0677093  -0.00554396  1.        ]]


In [9]:
if True:
    if cfg.MODEL.SPLIT == 'by-task':
        num_tasks = cfg.DATA.NUM_TASKS
    else:
        num_tasks = 1
    
    #num_tasks = cfg.DATA.NUM_TASKS
    num_outputs = cfg.DATA.NUM_OUTPUTS
    num_total_outputs = num_outputs*num_tasks
    arr_metrics = np.zeros((3,1+num_total_outputs)) #row:mae, r2, avg_std; col: total mean, task1(output1), task2(output2),...

    print('number of tasks that are being compared: ',num_total_outputs)

    best_idx = np.argmin(mean_arr_cv_mae[1,:])
    #best_idx = np.argmax(mean_arr_cv_r2[1,:])

    print('best_idx',best_idx)
    print('best combinations: ',combs[best_idx])

    arr_metrics[0,0] = np.min(mean_arr_cv_mae[1,:])
    arr_metrics[1,0] = np.max(mean_arr_cv_r2[1,:])
    arr_metrics[2,0] = mean_arr_cv_err[1,best_idx]

    print('best mean mae %.3f' % arr_metrics[0,0])
    print('best mean r2 %.3f' % arr_metrics[1,0])
    print('best mean err %.3f' % arr_metrics[2,0])
        
    cols_target_new = ['total']
    for task_ind in range(num_tasks):
        for out_ind in range(num_outputs):
            j = out_ind + num_outputs*task_ind
    #for j in range(num_total_outputs):
            j_name = 'Task_'+str(task_ind+1) + '_Output_'+str(out_ind+1)
            cols_target_new.append(j_name)
            arr_metrics[0,j+1] = ls_arr_cv_mae[j][1,best_idx]
            arr_metrics[1,j+1] = ls_arr_cv_r2[j][1,best_idx]
            arr_metrics[2,j+1] = ls_arr_cv_err[j][1,best_idx]
            print('best mae of target %d is %.3f' %(j,ls_arr_cv_mae[j][1,best_idx]))
            print('best r2 of target %d is %.3f' %(j,ls_arr_cv_r2[j][1,best_idx]))
            print('best err of target %d is %.3f' % (j,ls_arr_cv_err[j][1,best_idx]))
    
    df_test_metrics = pd.DataFrame(np.round(arr_metrics,3),columns = cols_target_new, 
                               index = [cfg.MODEL.METRIC,'R2','AVG_STD'])
    print(df_test_metrics)
    
    df_test_metrics.to_csv(setup_data.output_path+'/df_test_metrics.csv')

number of tasks that are being compared:  5
best_idx 0
best combinations:  (0.1, 150)
best mean mae 0.774
best mean r2 0.284
best mean err 32.871
best mae of target 0 is 0.440
best r2 of target 0 is 0.610
best err of target 0 is 0.632
best mae of target 1 is 0.935
best r2 of target 1 is 0.089
best err of target 1 is 0.438
best mae of target 2 is 0.133
best r2 of target 2 is 0.882
best err of target 2 is 51.950
best mae of target 3 is 1.123
best r2 of target 3 is -0.054
best err of target 3 is 26.731
best mae of target 4 is 1.239
best r2 of target 4 is -0.108
best err of target 4 is 84.607
          total  Task_1_Output_1  Task_1_Output_2  Task_1_Output_3  \
NMSE      0.774            0.440            0.935            0.133   
R2        0.284            0.610            0.089            0.882   
AVG_STD  32.871            0.632            0.438           51.950   

         Task_1_Output_4  Task_1_Output_5  
NMSE               1.123            1.239  
R2                -0.054           

## MTMO Cross validation

In [ ]:
def set_up_mtmo_cv(self):
        ls_model_from_combs = []

        ls_lr = [0.1,0.15,0.25]#[0.1,0.15,0.2,0.25,0.3,0.35,0.40]
        ls_n_iter = [500,700,900,1100]#[200,300,400,500,700]
        combs = list(itertools.product(ls_lr,ls_n_iter))

        mean_arr_cv_mae = np.zeros((2,len(combs))) #train,test
        mean_arr_cv_r2 = np.zeros((2,len(combs))) #train,test
        mean_arr_cv_err = np.zeros((2,len(combs))) #train,test

        ls_arr_cv_mae = [np.zeros((2,len(combs))) for _ in range(self.num_total_output)]
        ls_arr_cv_r2 = [np.zeros((2,len(combs))) for _ in range(self.num_total_output)]
        ls_arr_cv_err = [np.zeros((2,len(combs))) for _ in range(self.num_total_output)]

        for i in range(0,len(combs)):
            lr,n_iter = combs[i]
            arr_train_metrics = np.zeros((11,self.num_cols))
            arr_test_metrics = np.zeros((11,self.num_cols))
            pre_model = None
            for batch_ind in range(self.n_cv):
                t_train_x = self.ls_X_train[batch_ind]
                t_train_y = self.ls_y_train[batch_ind]
                t_test_x = self.ls_X_test[batch_ind]
                t_test_y = self.ls_y_test[batch_ind]

                likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(
                    num_tasks=self.num_outputs, 
                    rank=self.link_rank if self.link_rank is not None else 0,
                )
                model = MTMOGP(t_train_x, t_train_y,likelihood,
                               task_rank = self.num_tasks,output_rank = self.num_outputs)

                if batch_ind>0:
                    source_state_dict = pre_model.state_dict()
                    model.load_state_dict(source_state_dict)
                # Find optimal model hyperparameters
                model.train()
                likelihood.train()

                # Use the adam optimizer
                optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Includes GaussianLikelihood parameters

                # "Loss" for GPs - the marginal log likelihood
                mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

                training_iterations = n_iter#500
                for _ in range(training_iterations):
                    optimizer.zero_grad()
                    output = model(t_train_x)
                    loss = -mll(output, t_train_y)
                    loss.backward()
                    #print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
                    optimizer.step()

                ls_model_from_combs.append(model)
                if batch_ind == 0:
                    pre_model = model

                # Set into eval mode
                model.eval()
                likelihood.eval()

                arr_train_x = t_train_x.detach().numpy()
                arr_test_x = t_test_x.detach().numpy()
                with torch.no_grad(), gpytorch.settings.fast_pred_var():
                    test_pred = likelihood(model(t_test_x))   
                    test_mean = test_pred.mean
                    test_lower, test_upper = test_pred.confidence_region()
                    train_pred = likelihood(model(t_train_x))
                    train_mean = train_pred.mean
                    train_lower, train_upper = train_pred.confidence_region()
    
                if self.y_scale=='no-y-scale':
                    arr_train_y = t_train_y.detach().numpy()
                    arr_test_y = t_test_y.detach().numpy()
                    arr_test_mean = test_mean.detach().numpy()
                    arr_test_lower = test_lower.detach().numpy()
                    arr_test_upper = test_upper.detach().numpy()

                    arr_train_mean = train_mean.detach().numpy()
                    arr_train_lower = train_lower.detach().numpy()
                    arr_train_upper = train_upper.detach().numpy()
                else:
                    y_scaler = self.obj_y_scaler[batch_ind]
                    arr_train_y = y_scaler.inverse_transform(t_train_y)
                    arr_test_y = t_test_y.detach().numpy()

                    arr_train_mean = y_scaler.inverse_transform(train_mean)
                    arr_train_lower = y_scaler.inverse_transform(train_lower)
                    arr_train_upper = y_scaler.inverse_transform(train_upper)

                    arr_test_mean = y_scaler.inverse_transform(test_mean)
                    arr_test_lower = y_scaler.inverse_transform(test_lower)
                    arr_test_upper = y_scaler.inverse_transform(test_upper)

                for task_ind in range(self.num_tasks):
                    for j in range(self.num_outputs):
                        mtmo_ind = j + self.num_outputs*task_ind
                        row_train_inds = arr_train_x[:,-1]==task_ind
                        r_train_y = arr_train_y[row_train_inds,j]
                        r_train_mean_y =arr_train_mean[row_train_inds,j]
                        r_train_lower_y =arr_train_lower[row_train_inds,j]
                        r_train_upper_y =arr_train_upper[row_train_inds,j]
                        r_train_std = r_train_mean_y - r_train_lower_y

                        row_test_inds = arr_test_x[:,-1]==task_ind
                        r_test_y = arr_test_y[row_test_inds,j]
                        r_test_mean_y =arr_test_mean[row_test_inds,j]
                        r_test_lower_y =arr_test_lower[row_test_inds,j]
                        r_test_upper_y =arr_test_upper[row_test_inds,j]
                        r_test_std = r_test_mean_y - r_test_lower_y
    
                        train_comp = np.concatenate((r_train_y.reshape(-1,1),r_train_mean_y.reshape(-1,1)),axis = 1)
                        df_train_comp = pd.DataFrame(train_comp,columns = ['true','pred'])
                        df_train_comp['upper'] = r_train_upper_y
                        df_train_comp['lower'] = r_train_lower_y
                        df_train_comp['mode'] = 'train'
    
                        test_comp = np.concatenate((r_test_y.reshape(-1,1),r_test_mean_y.reshape(-1,1)),axis = 1)
                        df_test_comp = pd.DataFrame(test_comp,columns = ['true','pred'])
                        df_test_comp['upper'] = r_test_upper_y
                        df_test_comp['lower'] = r_test_lower_y
                        df_test_comp['mode'] = 'test'

                        y_true = r_test_y
                        y_pred = r_test_mean_y
                        y_train = r_train_y
                        mean_train = r_train_mean_y
    
                        a = batch_ind + self.n_cv*mtmo_ind
                        #a = mtmo_ind + self.num_total_output*batch_ind
                        arr_test_metrics[0,a] = metrics.mean_absolute_error(y_true, y_pred)
                        arr_test_metrics[1,a] = metrics.median_absolute_error(y_true, y_pred)
                        arr_test_metrics[2,a] = metrics.mean_squared_error(y_true, y_pred)
                        arr_test_metrics[3,a] = metrics.root_mean_squared_error(y_true, y_pred)
                        arr_test_metrics[4,a] = metrics.mean_absolute_percentage_error(y_true, y_pred)
                        arr_test_metrics[5,a] = metrics.max_error(y_true, y_pred)
                        arr_test_metrics[6,a] = metrics.explained_variance_score(y_true, y_pred)
                        arr_test_metrics[7,a] = metrics.r2_score(y_true, y_pred)
                        arr_test_metrics[8,a] = np.mean(r_test_std)
                        arr_test_metrics[9,a] = np.min(r_test_std)
                        arr_test_metrics[10,a] = np.max(r_test_std)

                        arr_train_metrics[0,a] = metrics.mean_absolute_error(y_train, mean_train)
                        arr_train_metrics[1,a] = metrics.median_absolute_error(y_train, mean_train)
                        arr_train_metrics[2,a] = metrics.mean_squared_error(y_train, mean_train)
                        arr_train_metrics[3,a] = metrics.root_mean_squared_error(y_train, mean_train)
                        arr_train_metrics[4,a] = metrics.mean_absolute_percentage_error(y_train, mean_train)
                        arr_train_metrics[5,a] = metrics.max_error(y_train, mean_train)
                        arr_train_metrics[6,a] = metrics.explained_variance_score(y_train, mean_train)
                        arr_train_metrics[7,a] = metrics.r2_score(y_train, mean_train)
                        arr_train_metrics[8,a] = np.mean(r_train_std)
                        arr_train_metrics[9,a] = np.min(r_train_std)
                        arr_train_metrics[10,a] = np.max(r_train_std)
                        arr_train_metrics[8,a] = np.mean(r_train_std)
                        arr_train_metrics[9,a] = np.min(r_train_std)
                        arr_train_metrics[10,a] = np.max(r_train_std)
      
                        ls_arr_cv_mae[mtmo_ind][0,i] = np.mean(arr_train_metrics[self.metric_row,self.n_cv*mtmo_ind:self.n_cv*(mtmo_ind+1)])
                        ls_arr_cv_mae[mtmo_ind][1,i] = np.mean(arr_test_metrics[self.metric_row,self.n_cv*mtmo_ind:self.n_cv*(mtmo_ind+1)])

                        ls_arr_cv_r2[mtmo_ind][0,i] = np.mean(arr_train_metrics[6,self.n_cv*mtmo_ind:self.n_cv*(mtmo_ind+1)])
                        ls_arr_cv_r2[mtmo_ind][1,i] = np.mean(arr_test_metrics[6,self.n_cv*mtmo_ind:self.n_cv*(mtmo_ind+1)])

                        ls_arr_cv_err[mtmo_ind][0,i] = np.mean(arr_train_metrics[8,self.n_cv*mtmo_ind:self.n_cv*(mtmo_ind+1)])
                        ls_arr_cv_err[mtmo_ind][1,i] = np.mean(arr_test_metrics[8,self.n_cv*mtmo_ind:self.n_cv*(mtmo_ind+1)])

            mean_arr_cv_mae[0,i] = np.mean(arr_train_metrics[self.metric_row,:])
            mean_arr_cv_mae[1,i] = np.mean(arr_test_metrics[self.metric_row,:])

            mean_arr_cv_r2[0,i] = np.mean(arr_train_metrics[6,:])
            mean_arr_cv_r2[1,i] = np.mean(arr_test_metrics[6,:])

            mean_arr_cv_err[0,i] = np.mean(arr_train_metrics[8,:])
            mean_arr_cv_err[1,i] = np.mean(arr_test_metrics[8,:])


        return combs,ls_model_from_combs,ls_arr_cv_mae,ls_arr_cv_r2,ls_arr_cv_err,mean_arr_cv_mae,mean_arr_cv_r2,mean_arr_cv_err

In [ ]:
if True:
    mtmo_cv_trainer = CV_Trainer(cfg,ls_X_train,ls_X_test,ls_y_train,ls_y_test,obj_y_scaler)
    combs,ls_model_from_combs,ls_arr_cv_mae,ls_arr_cv_r2,ls_arr_cv_err,mean_arr_cv_mae,mean_arr_cv_r2,mean_arr_cv_err = mtmo_cv_trainer.set_up_cv_model()

    if cfg.MODEL.SPLIT == 'by-task':
        num_tasks = cfg.DATA.NUM_TASKS
    else:
        num_tasks = 1
    num_outputs = cfg.DATA.NUM_OUTPUTS
    num_total_outputs = num_outputs*num_tasks
    arr_metrics = np.zeros((3,1+num_total_outputs)) #row:mae, r2, avg_std; col: total mean, task1(output1), task2(output2),...

    print('number of tasks that are being compared: ',num_total_outputs)

    best_idx = np.argmin(mean_arr_cv_mae[1,:])
    #best_idx = np.argmax(mean_arr_cv_r2[1,:])

    print('best_idx',best_idx)
    print('best combinations: ',combs[best_idx])

    arr_metrics[0,0] = np.min(mean_arr_cv_mae[1,:])
    arr_metrics[1,0] = np.max(mean_arr_cv_r2[1,:])
    arr_metrics[2,0] = mean_arr_cv_err[1,best_idx]

    print('best mean mae %.3f' % arr_metrics[0,0])
    print('best mean r2 %.3f' % arr_metrics[1,0])
    print('best mean err %.3f' % arr_metrics[2,0])
    
    cols_target_new = ['total']
    #for j in range(cfg.DATA.NUM_OUTPUTS):
    for task_ind in range(num_tasks):
        for out_ind in range(num_outputs):
            j = out_ind + num_outputs*task_ind
            j_name = 'Task_'+str(task_ind+1) + '_Output_'+str(out_ind+1)
            cols_target_new.append(j_name)
            arr_metrics[0,j+1] = ls_arr_cv_mae[j][1,best_idx]
            arr_metrics[1,j+1] = ls_arr_cv_r2[j][1,best_idx]
            arr_metrics[2,j+1] = ls_arr_cv_err[j][1,best_idx]
            print('best mae of target %d is %.3f' %(j,ls_arr_cv_mae[j][1,best_idx]))
            print('best r2 of target %d is %.3f' %(j,ls_arr_cv_r2[j][1,best_idx]))
            print('best err of target %d is %.3f' % (j,ls_arr_cv_err[j][1,best_idx]))
    
    df_test_metrics = pd.DataFrame(np.round(arr_metrics,3),columns = cols_target_new, 
                               index = [cfg.MODEL.METRIC,'R2','AVG_STD'])
    print(df_test_metrics)
    
    output_path = '/'.join([cfg.PATH.RESULT,cfg.MISC.DATA_NAME,cfg.MODEL.MODEL_NAME,cfg.MODEL.SPLIT,cfg.MISC.DATE,'num_tasks_'+str(cfg.DATA.NUM_TASKS)])
    df_test_metrics.to_csv(output_path+'/df_test_metrics.csv')

## MTMO-LMGP CV

In [ ]:
if True:
    mtmolmgp_cv_trainer = CV_Trainer(cfg,ls_X_train,ls_X_test,ls_y_train,ls_y_test,obj_y_scaler)
    combs,ls_model_from_combs,ls_arr_cv_mae,ls_arr_cv_r2,ls_arr_cv_err,mean_arr_cv_mae,mean_arr_cv_r2,mean_arr_cv_err = mtmolmgp_cv_trainer.set_up_cv_model()

    if cfg.MODEL.SPLIT == 'by-task':
        num_tasks = cfg.DATA.NUM_TASKS
    else:
        num_tasks = 1
    num_outputs = cfg.DATA.NUM_OUTPUTS
    num_total_outputs = num_outputs*num_tasks
    arr_metrics = np.zeros((3,1+num_total_outputs)) #row:mae, r2, avg_std; col: total mean, task1(output1), task2(output2),...

    print('number of tasks that are being compared: ',num_total_outputs)

    best_idx = np.argmin(mean_arr_cv_mae[1,:])
    #best_idx = np.argmax(mean_arr_cv_r2[1,:])

    print('best_idx',best_idx)
    print('best combinations: ',combs[best_idx])

    arr_metrics[0,0] = np.min(mean_arr_cv_mae[1,:])
    arr_metrics[1,0] = np.max(mean_arr_cv_r2[1,:])
    arr_metrics[2,0] = mean_arr_cv_err[1,best_idx]

    print('best mean mae %.3f' % arr_metrics[0,0])
    print('best mean r2 %.3f' % arr_metrics[1,0])
    print('best mean err %.3f' % arr_metrics[2,0])
    
    cols_target_new = ['total']
    #for j in range(cfg.DATA.NUM_OUTPUTS):
    for task_ind in range(num_tasks):
        for out_ind in range(num_outputs):
            j = out_ind + num_outputs*task_ind
            j_name = 'Task_'+str(task_ind+1) + '_Output_'+str(out_ind+1)
            cols_target_new.append(j_name)
            arr_metrics[0,j+1] = ls_arr_cv_mae[j][1,best_idx]
            arr_metrics[1,j+1] = ls_arr_cv_r2[j][1,best_idx]
            arr_metrics[2,j+1] = ls_arr_cv_err[j][1,best_idx]
            print('best mae of target %d is %.3f' %(j,ls_arr_cv_mae[j][1,best_idx]))
            print('best r2 of target %d is %.3f' %(j,ls_arr_cv_r2[j][1,best_idx]))
            print('best err of target %d is %.3f' % (j,ls_arr_cv_err[j][1,best_idx]))
    
    df_test_metrics = pd.DataFrame(np.round(arr_metrics,3),columns = cols_target_new, 
                               index = ['MAE','R2','AVG_STD'])
    print(df_test_metrics)
    
    output_path = '/'.join([cfg.PATH.RESULT,cfg.MISC.DATA_NAME,cfg.MODEL.MODEL_NAME,cfg.MODEL.SPLIT,cfg.MISC.DATE,'num_tasks_'+str(cfg.DATA.NUM_TASKS)])
    df_test_metrics.to_csv(output_path+'/df_test_metrics.csv')